In [19]:
import napari
import btrack
import numpy as np
from skimage.io import imread
from btrack.utils import tracks_to_napari
print("version no.:", napari.__version__)

def find_apoptosis_index_time(target_track):
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    return apop_index, apop_time

def find_apoptosis_index(target_track):
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    return apop_index

version no.: 0.4.1


In [20]:
gfp = imread('/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/stacks/gfp.tif')

In [21]:
rfp = imread('/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/stacks/rfp.tif')

In [22]:
gfp_raw = imread('/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/stacks/gfp_raw.tif')

In [23]:
rfp_raw = imread('/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/stacks/rfp_raw.tif')

In [ ]:
#bf = imread('/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/stacks/bf.tif')

In [24]:
with btrack.dataio.HDF5FileHandler("/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5", 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler("/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5", 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks

[INFO][2020/11/27 06:53:11 pm] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
INFO:worker_process:Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/11/27 06:53:12 pm] Loading tracks/obj_type_1
INFO:worker_process:Loading tracks/obj_type_1
[INFO][2020/11/27 06:53:15 pm] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
INFO:worker_process:Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2020/11/27 06:53:20 pm] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
INFO:worker_process:Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2020/11/27 06:53:20 pm] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
INFO:worker_process:Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV08

In [25]:
divisions = [t for t in wt_tracks if t.fate.name == "DIVIDE" ]#and t.in_frame(730)]

In [26]:
scr_apops = [t for t in scr_tracks if t.fate.name == "APOPTOSIS"]

In [27]:
### isolate one target scribble track of interest
print("input scribble track of interest ID")
cell_ID = int(input())
index = [i for i, scr_apop in enumerate(scr_apops) if scr_apop.ID == cell_ID][0]
target_track = scr_apops[index]##### Show the first classification of each track
apop_index, apop_time = find_apoptosis_index_time(target_track)

input scribble track of interest ID
17


In [28]:
target_tracks = [target_track for i in (1,2)]

In [29]:
wt_data, properties, graph = tracks_to_napari(divisions, ndim = 2)

In [30]:
scr_data, properties, graph = tracks_to_napari(scr_apops, ndim = 2)

temp_data is just the target_track (Scr apop 17) 

In [ ]:
temp_data, properties, graph = tracks_to_napari(target_tracks, ndim = 2)

In [ ]:
target_track

`x = temp_data[:,3], y = temp_data[:,2]` this is how the tracks_to_napari organises itself

# ISSUES
#### is this corresponding to an overall shift of +20, +38? raw coords are 1688x1352

In [35]:
print(shift_y)

76


#### does this actually switch the coords? i think sooooo...

In [38]:
t,y,x = gfp_raw.shape
shift_x = int((x - 1600)/2)
shift_y = int((y - 1200)/2)

# tmp = wt_data[:,2].copy()
# wt_data[:,2] = wt_data[:,3]
# wt_data[:,3] = tmp
tmp = scr_data[:,2].copy()
scr_data[:,2] = scr_data[:,3]
scr_data[:,3] = tmp

# wt_data[:,3] = wt_data[:,3] #+ shift_y
# wt_data[:,2] = wt_data[:,2] #+ shift_x

### BUT I SWITCH THE AXIS AROUND!!! hence shift_x needs to be added to y

scr_data[:,3] = scr_data[:,3] #+ shift_x
scr_data[:,2] = scr_data[:,2] #+ shift_y

#### issue: coords seem to be jumbled but switching them using the above technique does not work 

In [ ]:
scr_data[:,3]

#### Need to provide coords to napari as [txy]

In [ ]:
apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_x, target_track.y[apop_index]+shift_y

In [ ]:
cell_IDs = [1, 3, 4, 6, 10]
apop_event = [None]* (max(cell_IDs)+1)
for ID in cell_IDs:
    index = [i for i, scr_apop in enumerate(scr_apops) if scr_apop.ID == ID][0]
    target_track = scr_apops[index]##### Show the first classification of each track
    apop_index, apop_time = find_apoptosis_index_time(target_track)

    apop_event[ID] = target_track.t[apop_index], target_track.x[apop_index]+shift_x, target_track.y[apop_index]+shift_y
   
print("list_of_apop_time_coords", [apop_event[i][0] for i in cell_IDs])

In [39]:
with napari.gui_qt():
    viewer = napari.Viewer()
    #viewer.add_image(bf)
    viewer.add_image(gfp_raw, name="gfp", blending = "additive", colormap = "green")
    viewer.add_image(rfp_raw, name="rfp", blending = "additive", colormap = "red")
    #viewer.add_points(data[:,1:])
    viewer.add_tracks(wt_data)
    viewer.add_tracks(scr_data)
    #viewer.add_points((0,19, 20))
#     viewer.add_points(apop_event[1])
#     viewer.add_points(apop_event[3])
#     viewer.add_points(apop_event[4])
#     viewer.add_points(apop_event[6])
#     viewer.add_points(apop_event[10])